In [121]:
import requests
import pandas as pd


API_KEY = "ca72e792bc6e44b4be1230936250904"
CITY = "Lahore"  
URL = f"http://api.weatherapi.com/v1/history.json?key={API_KEY}&q={CITY}&dt="

# Example to fetch data for the last 30 days (you can adjust this)
dates = pd.date_range(end=pd.to_datetime('today'), periods=30).strftime('%Y-%m-%d')

weather_data = []
for date in dates:
    response = requests.get(URL + date)
    data = response.json()
    if 'forecast' in data:
        for hour in data['forecast']['forecastday'][0]['hour']:
            weather_data.append({
                'time': hour['time'],
                'temp': hour['temp_c'],
                'humidity': hour['humidity'],
                'pressure': hour['pressure_mb'],
                'visibility': hour['vis_km'],
                'cloud_cover': hour['cloud']
            })

weather_df = pd.DataFrame(weather_data)

# Display the fetched data
print(weather_df.head())


               time  temp  humidity  pressure  visibility  cloud_cover
0  2025-03-12 00:00  19.9        38    1012.0        10.0           11
1  2025-03-12 01:00  18.7        42    1011.0        10.0            7
2  2025-03-12 02:00  18.1        43    1010.0        10.0            8
3  2025-03-12 03:00  17.6        45    1010.0        10.0            9
4  2025-03-12 04:00  17.4        46    1010.0        10.0           10


In [122]:
def categorize_temperature(temp):
    if temp < 15:
        return 'Low'
    elif 15 <= temp <= 30:
        return 'Medium'
    else:
        return 'High'

def categorize_humidity(humidity):
    if humidity < 30:
        return 'Low'
    elif 30 <= humidity <= 70:
        return 'Medium'
    else:
        return 'High'


def categorize_pressure(pressure):
    if pressure < 1000:
        return 'Low'
    elif 1000 <= pressure <= 1020:
        return 'Medium'
    else:
        return 'High'

def categorize_visibility(visibility):
    if visibility < 5:
        return 'Poor'
    elif 5 <= visibility <= 15:
        return 'Moderate'
    else:
        return 'Good'

def categorize_cloud_cover(cloud_cover):
    if cloud_cover < 25:
        return 'Clear'
    elif 25 <= cloud_cover <= 75:
        return 'Partly Cloudy'
    else:
        return 'Overcast'

# Apply categorization functions
weather_df['temp_cat'] = weather_df['temp'].apply(categorize_temperature)
weather_df['humidity_cat'] = weather_df['humidity'].apply(categorize_humidity)
weather_df['pressure_cat'] = weather_df['pressure'].apply(categorize_pressure)
weather_df['visibility_cat'] = weather_df['visibility'].apply(categorize_visibility)
weather_df['cloud_cover_cat'] = weather_df['cloud_cover'].apply(categorize_cloud_cover)

# Display the categorized data
print(weather_df[['time', 'temp_cat', 'humidity_cat','pressure_cat', 'visibility_cat', 'cloud_cover_cat']].head())


               time temp_cat humidity_cat pressure_cat visibility_cat  \
0  2025-03-12 00:00   Medium       Medium       Medium       Moderate   
1  2025-03-12 01:00   Medium       Medium       Medium       Moderate   
2  2025-03-12 02:00   Medium       Medium       Medium       Moderate   
3  2025-03-12 03:00   Medium       Medium       Medium       Moderate   
4  2025-03-12 04:00   Medium       Medium       Medium       Moderate   

  cloud_cover_cat  
0           Clear  
1           Clear  
2           Clear  
3           Clear  
4           Clear  


In [123]:
import numpy as np

def create_transition_matrix(data_column):
    unique_states = data_column.unique()
    transition_matrix = pd.DataFrame(np.zeros((len(unique_states), len(unique_states))), columns=unique_states, index=unique_states)
    
    for i in range(1, len(data_column)):
        current_state = data_column[i-1]
        next_state = data_column[i]
        transition_matrix.at[current_state, next_state] += 1
    
    transition_matrix = transition_matrix.div(transition_matrix.sum(axis=1), axis=0)
    
    return transition_matrix

humidity_transition_matrix = create_transition_matrix(weather_df['humidity_cat'])


def smooth_transition_matrix(transition_matrix, alpha=1):
    # Add smoothing (Laplace smoothing) to avoid zero probabilities
    smoothed_matrix = transition_matrix + alpha
    smoothed_matrix = smoothed_matrix.div(smoothed_matrix.sum(axis=1), axis=0)
    return smoothed_matrix

humidity_transition_matrix_smooth= smooth_transition_matrix(humidity_transition_matrix, alpha=1)


print(humidity_transition_matrix_smooth)




          Medium       Low     High
Medium  0.466000  0.278000  0.25600
Low     0.255702  0.494298  0.25000
High    0.281250  0.250000  0.46875


In [124]:
def predict_next_state(current_state, transition_matrix):
    state_probs = transition_matrix.loc[current_state]
    
    if state_probs.sum() == 0:
        return np.random.choice(transition_matrix.columns)
    
    return state_probs.idxmax()

current_temp_state = weather_df['temp_cat'].iloc[-1]  
predicted_temp = predict_next_state(current_temp_state, humidity_transition_matrix_smooth)

def predict_next_state_probabilistic(current_state, transition_matrix):
    state_probs = transition_matrix.loc[current_state]
    
    if state_probs.sum() == 0:
        return np.random.choice(transition_matrix.columns)
    
    return np.random.choice(state_probs.index, p=state_probs.values)

predicted_temp = predict_next_state_probabilistic(current_temp_state, humidity_transition_matrix_smooth)
print(f"Predicted Humidity: {predicted_temp}")
 


Predicted Humidity: Medium


In [125]:
def predict_n_steps_probabilistic(current_state, transition_matrix, n_steps):
    state = current_state
    predictions = []
    for _ in range(n_steps):
        state_probs = transition_matrix.loc[state]
        
        if state_probs.sum() == 0:
            state = np.random.choice(transition_matrix.columns)
        else:
            state = np.random.choice(state_probs.index, p=state_probs.values)
        
        predictions.append(state)
    
    return predictions


In [126]:
current_temp_state = weather_df['humidity_cat'].iloc[-1]

predicted_states = predict_n_steps_probabilistic(current_temp_state, humidity_transition_matrix_smooth, 3)

print("Humidity Forecast:")
print(f"After 2 hours: {predicted_states[0]}")
print(f"After 4 hours: {predicted_states[1]}")
print(f"After 6 hours: {predicted_states[2]}")


Humidity Forecast:
After 2 hours: High
After 4 hours: Low
After 6 hours: Low
